In [1]:
from copy import deepcopy
from tqdm import tqdm

import sys
sys.path.insert(0, "../")

from src.db_drivers.vector_driver.connectors import MilvusConnector, DEFAULT_MILVUS_CONFIG
from src.db_drivers.vector_driver import VectorDBInstance, VectorDriverConfig, VectorDBConnectionConfig, VectorDriver
from src.db_drivers.graph_driver import GraphDriverConfig, GraphDBConnectionConfig
from src.db_drivers.graph_driver.configs import DEFAULT_NEO4J_CONFIG

from src.pipelines.memorize import MemPipeline, MemPipelineConfig
from src.pipelines.memorize.MemPipeline import LLMExtractorConfig, LLMUpdatorConfig

from src.kg_model import KnowledgeGraphModel, GraphModelConfig, EmbedderModelConfig, EmbeddingsModelConfig

from src.pipelines.qa.kg_reasoning import KnowledgeGraphReasonerConfig
from src.pipelines.qa.kg_reasoning.weak_reasoner.knowledge_retriever import WaterCirclesSearchConfig, WaterCirclesTripletsRetriever
from src.pipelines.qa.kg_reasoning.weak_reasoner import QueryLLMParserConfig, KnowledgeComparatorConfig, \
    KnowledgeRetrieverConfig, QALLMGeneratorConfig, WeakKGReasonerConfig

from src.pipelines.qa import QAPipeline, QAPipelineConfig

from src.agents import AgentDriver, AgentDriverConfig
from src.agents.connectors.OpenAIConnector import DEEPSEEK_CONFIG, GPT4OMINI_CONFIG

/home/dzigen/Desktop/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pickle 
import hashlib

v = {'a': 2, 'b': 'c'}
d = pickle.dumps(v)
hashlib.sha1(d).hexdigest()

'3f813fbd0510c6941817d2bb97d078384ac28baf'

In [4]:
'3f813fbd0510c6941817d2bb97d078384ac28baf'

'3f813fbd0510c6941817d2bb97d078384ac28baf'

In [2]:
from src.db_drivers.vector_driver.connectors.MilvusConnector import MilvusConnector, DEFAULT_MILVUS_CONFIG

In [37]:
DEFAULT_MILVUS_CONFIG.conn['port'] = 19534
DEFAULT_MILVUS_CONFIG.db_info['db'] = 'knowledgegraphdb'
DEFAULT_MILVUS_CONFIG.db_info['table'] = 'vectorized_triplets'
DEFAULT_MILVUS_CONFIG.params['load'] = False

In [38]:
conn = MilvusConnector(DEFAULT_MILVUS_CONFIG)

In [ ]:
conn.close_connection()

In [14]:
DEFAULT_MILVUS_CONFIG.params['load'] = False

In [39]:
conn.open_connection()

In [41]:
conn.client.list_indexes('vectorized_triplets')

['id_index', 'embedding_index']

In [43]:
conn.client.release_collection('vectorized_triplets')

In [44]:
conn.client.drop_index(collection_name='vectorized_triplets', index_name='embedding_index')

In [19]:
import numpy as np

In [20]:
query = VectorDBInstance(embedding=np.random.rand(1024))

In [26]:
conn.retrieve([query])

[[(0.6257064342498779,
   VectorDBInstance(id='51185aed020ecd5d79ebf77bf14f62c1', document='looks like', embedding=[0.04149831458926201, 0.014635846950113773, -0.012789545580744743, -0.03465275093913078, 0.04347336292266846, -0.02498744986951351, -0.01049241703003645, 0.06890533864498138, 0.049556925892829895, -0.016313940286636353, 0.02287660352885723, 0.045745350420475006, -0.03777341544628143, -0.04441259801387787, -0.04542023688554764, -0.03941377252340317, 0.0012623362708836794, 0.047332391142845154, 0.019105298444628716, 0.013023152947425842, 0.030441707000136375, 0.0037172117736190557, -0.04607018455862999, -0.05041733756661415, -0.03293001279234886, -0.010852385312318802, -0.03638295456767082, -0.03985891491174698, -0.015228703618049622, -0.023905033245682716, -0.023959867656230927, 0.004089282359927893, -0.03530096635222435, 7.6270654972177e-05, -0.01355139072984457, -0.001282126409932971, 0.03563676029443741, 0.006578616797924042, -0.024406196549534798, 0.052304238080978394, 

In [18]:
conn.count_items()

48882

In [2]:
from src.db_drivers.graph_driver.connectors.Neo4jConnector import Neo4jConnector, DEFAULT_NEO4J_CONFIG

In [3]:
DEFAULT_NEO4J_CONFIG

GraphDBConnectionConfig(db_info={'db': 'personalaidb', 'table': 'personalaitable'}, params={'user': 'neo4j', 'pwd': 'password'}, need_to_clear=False, host='localhost', port=7687)

In [4]:
conn = Neo4jConnector()

In [9]:
from src.utils.data_structs import TripletCreator, NodeCreator, RelationCreator, NodeType

In [7]:
conn.create(
    triplets=[TripletCreator.create(
        start_node=NodeCreator.create(n_type='object', name='123'),
        relation=RelationCreator.create(r_type='simple', name='456'),
        end_node=NodeCreator.create(n_type='object', name='890'))]
)

In [10]:
conn.read_by_name(name='123', object_type=NodeType.object, object='node')

[Node(name='123', type=<NodeType.object: 'object'>, prop={'name': '123', 'str_id': '202cb962ac59075b964b07152d234b70'}, stringified='123', id='202cb962ac59075b964b07152d234b70')]

In [5]:
conn.open_connection()

In [5]:
conn.close_connection()

In [2]:
graph_config = GraphModelConfig(
    driver_config=GraphDriverConfig(
        db_vendor='neo4j',
        db_config=DEFAULT_NEO4J_CONFIG))

graph_config.driver_config.db_config.need_to_clear = True

embeddings_config = EmbeddingsModelConfig(
    nodesdb_driver_config=VectorDriverConfig(
        db_vendor='milvus', db_config=deepcopy(DEFAULT_MILVUS_CONFIG)),
    tripletsdb_driver_config=VectorDriverConfig(
        db_vendor='milvus',db_config=deepcopy(DEFAULT_MILVUS_CONFIG)),
    embedder_config=EmbedderModelConfig())

embeddings_config.nodesdb_driver_config.db_config.db_info['table'] = 'nodes_table'
embeddings_config.nodesdb_driver_config.db_config.params['vector_dim'] = 384
embeddings_config.nodesdb_driver_config.db_config.need_to_clear = True

embeddings_config.tripletsdb_driver_config.db_config.db_info['table'] = 'triplets_table'
embeddings_config.tripletsdb_driver_config.db_config.params['vector_dim'] = 384
embeddings_config.tripletsdb_driver_config.db_config.need_to_clear = True

In [3]:
kg_model = KnowledgeGraphModel(graph_config, embeddings_config, verbose=True)

No sentence-transformers model found with name ../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [4]:
AGENT_DRIVER = AgentDriverConfig(name='openai',agent_config=GPT4OMINI_CONFIG)

In [5]:
mem_config = MemPipelineConfig(
    verbose=True,
    extractor_config=LLMExtractorConfig(
        verbose=True, lang='en',
        adriver_config=AGENT_DRIVER),
    updator_config=LLMUpdatorConfig(
        adriver_config=AGENT_DRIVER))

In [6]:
memorize_pipeline = MemPipeline(kg_model, mem_config)

In [7]:
messages = [
    "Mikhail Menshchikov is currently a second-year master's student at ITMO.",
    "Mikhail Menshchikov is studying in the Master's program 'Deep Learning and Generative AI'",
    "Mikhail Menshchikov completed his bachelor's degree at Petrozavodsk State University",
    "Petrozavodsk State University is where Mikhail Menshchikov received his bachelor's degree.",
    "Mikhail Menshchikov studied at Petrozavodsk State University and received a bachelor's degree."]
properties = [dict() for _ in range(len(messages))]

In [12]:
print(kg_model.graph_struct.count_items())
print(kg_model.embeddings_struct.count_items())

{'triplets': 37, 'nodes': 17}
{'nodes': 17, 'triplets': 14}


In [10]:
for text, prop in tqdm(list(zip(messages, properties))):
    _, info = memorize_pipeline.remember(text=text, properties=prop)

  0%|          | 0/5 [00:00<?, ?it/s]

START KNOWLEDGE REMEMBERING...
BASE_TEXT ID: 208f7b79f68f594eb4b9a9ced69b4b30
STAGE#1 - 'Извлечение информации (в структурированном формате) из текста'
START KNOWLEDGE EXTRACTION...
BASE_TEXT ID: 208f7b79f68f594eb4b9a9ced69b4b30
START SIMPLE-TRIPLETS EXTRACTION...
STATUS: Операция выполнена успешно.
RESULT: 2
* Triplet(start_node=Node(name='mikhail menshchikov', type=<NodeType.object: 'object'>, prop={}, stringified='mikhail menshchikov', id='0aa08329fe5ec36d1c6781ea2484d03e'), relation=Relation(name='is', type=<RelationType.simple: 'simple'>, prop={}, id='9d657afbd18c3d3668c0dfd8ca4a12a1'), end_node=Node(name="second-year master's student", type=<NodeType.object: 'object'>, prop={}, stringified="second-year master's student", id='19bfbff3b07ce5359ef31a2790a2a8a8'), stringified="mikhail menshchikov is second-year master's student", id='05a7fd88c8837c306c3aac058f026a4e')
* Triplet(start_node=Node(name='mikhail menshchikov', type=<NodeType.object: 'object'>, prop={}, stringified='mikhail

 20%|██        | 1/5 [00:10<00:42, 10.65s/it]

GRAPH DB STATUS: {'triplets': 8, 'nodes': 5}
VECTOR DB STATUS: 5 - nodes; 3 - triplets
STATUS: Операция выполнена успешно.
START KNOWLEDGE REMEMBERING...
BASE_TEXT ID: 69700ddb97181c48cdfc3f5cb3730be8
STAGE#1 - 'Извлечение информации (в структурированном формате) из текста'
START KNOWLEDGE EXTRACTION...
BASE_TEXT ID: 69700ddb97181c48cdfc3f5cb3730be8
START SIMPLE-TRIPLETS EXTRACTION...
STATUS: Операция выполнена успешно.
RESULT: 2
* Triplet(start_node=Node(name='mikhail menshchikov', type=<NodeType.object: 'object'>, prop={}, stringified='mikhail menshchikov', id='0aa08329fe5ec36d1c6781ea2484d03e'), relation=Relation(name='is studying in', type=<RelationType.simple: 'simple'>, prop={}, id='35d2b261ad6915d18df772fda25feb5d'), end_node=Node(name="master's program", type=<NodeType.object: 'object'>, prop={}, stringified="master's program", id='226111254fe63aef39a80483a05672e8'), stringified="mikhail menshchikov is studying in master's program", id='f47dcf6a14a0c149e8fa4c44405f9a05')
* Trip

 40%|████      | 2/5 [00:18<00:27,  9.27s/it]

...

GRAPH DB STATUS: {'triplets': 21, 'nodes': 12}
VECTOR DB STATUS: 12 - nodes; 9 - triplets
STATUS: Операция выполнена успешно.
START KNOWLEDGE REMEMBERING...
BASE_TEXT ID: a67f0ca6edf1709267f4e0095f8ad9c1
STAGE#1 - 'Извлечение информации (в структурированном формате) из текста'
START KNOWLEDGE EXTRACTION...
BASE_TEXT ID: a67f0ca6edf1709267f4e0095f8ad9c1
START SIMPLE-TRIPLETS EXTRACTION...
STATUS: Операция выполнена успешно.
RESULT: 2
* Triplet(start_node=Node(name='petrozavodsk state university', type=<NodeType.object: 'object'>, prop={}, stringified='petrozavodsk state university', id='1ebe389e62495d1ac6f5198d0fc61504'), relation=Relation(name='is where', type=<RelationType.simple: 'simple'>, prop={}, id='562137bd8d6ea97e60e0b95c654e6587'), end_node=Node(name="mikhail menshchikov received bachelor's degree", type=<NodeType.object: 'object'>, prop={}, stringified="mikhail menshchikov received bachelor's degree", id='9ccb4e3ceffed1c4f90146047586a4a6'), stringified="petrozavodsk state uni

 80%|████████  | 4/5 [00:39<00:09,  9.78s/it]

GRAPH DB STATUS: {'triplets': 31, 'nodes': 15}
VECTOR DB STATUS: 15 - nodes; 13 - triplets
STATUS: Операция выполнена успешно.
START KNOWLEDGE REMEMBERING...
BASE_TEXT ID: f2ee677eab441c35d161d402745ea25a
STAGE#1 - 'Извлечение информации (в структурированном формате) из текста'
START KNOWLEDGE EXTRACTION...
BASE_TEXT ID: f2ee677eab441c35d161d402745ea25a
START SIMPLE-TRIPLETS EXTRACTION...
STATUS: Операция выполнена успешно.
RESULT: 2
* Triplet(start_node=Node(name='mikhail menshchikov', type=<NodeType.object: 'object'>, prop={}, stringified='mikhail menshchikov', id='0aa08329fe5ec36d1c6781ea2484d03e'), relation=Relation(name='studied at', type=<RelationType.simple: 'simple'>, prop={}, id='ee9636f5aa793cb2da36f5981ce95ba1'), end_node=Node(name='petrozavodsk state university', type=<NodeType.object: 'object'>, prop={}, stringified='petrozavodsk state university', id='1ebe389e62495d1ac6f5198d0fc61504'), stringified='mikhail menshchikov studied at petrozavodsk state university', id='4f9144

100%|██████████| 5/5 [00:50<00:00, 10.06s/it]

GRAPH DB STATUS: {'triplets': 37, 'nodes': 17}
VECTOR DB STATUS: 17 - nodes; 14 - triplets
STATUS: Операция выполнена успешно.


In [11]:
RETRIEVER_NAME = 'watercircles'
RETRIEVER_HYPERP = WaterCirclesSearchConfig(
    strict_filter = True, hyper_num = 15,
    episodic_num = 15, chain_triplets_num = 25,
    other_triplets_num = 6, do_text_pruning = False)

In [13]:
LANGUAGE = 'en' # 'ru' | 'en' | 'auto'
# QA-pipeline configuration
QA_PIPELINE_CONFIG = QAPipelineConfig(
    verbose=True,
    reasoner_config=KnowledgeGraphReasonerConfig(
        verbose=True,
        reasoner_name='weak',
        reasoner_hyperparameters=WeakKGReasonerConfig(
            verbose=True,
            query_parser_config=QueryLLMParserConfig(lang=LANGUAGE, adriver_config=AGENT_DRIVER),
            knowledge_comparator_config=KnowledgeComparatorConfig(verbose=True),
            knowledge_retriever_config=KnowledgeRetrieverConfig(retriever_method=RETRIEVER_NAME,retriever_config=RETRIEVER_HYPERP),
            answer_generator_config=QALLMGeneratorConfig(lang=LANGUAGE, adriver_config=AGENT_DRIVER))))

In [14]:
qa_pipeline = QAPipeline(kg_model=kg_model, config=QA_PIPELINE_CONFIG)

In [ ]:
answer, info = qa_pipeline.answer("What program is Mikhail Menshchikov studying for his master's degree?")
print(answer)

-------------------------

In [3]:
config = VectorDriverConfig(db_vendor='milvus', db_config=VectorDBConnectionConfig(
        conn={'host': 'localhost', 'port': 19530, 'user': 'root', 'pass': 'Milvus'},
        db_info={'db': 'testing', 'table': 'testing'}, need_to_clear=True,
        params={'id_length': 32, 'vector_dim': 3, 'document_max_length': 51200, 
                'load': True, 'flush': False}))
conn = VectorDriver.connect(config)

In [4]:
instances = [
    VectorDBInstance(id='123', document='qwerty', embedding=[0.1, 0.2, 0.3], metadata={'k1': 'v1'}), 
    VectorDBInstance(id='456', document='ytrewq', embedding=[0.4, 0.5, 0.6], metadata={'k2': 'v2'})]

In [22]:
from time import sleep

In [23]:
conn.clear()
conn.create(instances)
sleep(1)
conn.item_exist('123')

data: []
data: []
items to add: 2
{'insert_count': 2, 'ids': ['123', '456']}
data: ["{'document': 'qwerty', 'metadata': {'k1': 'v1'}, 'id': '123', 'embedding': [0.1, 0.2, 0.3]}"]


True

In [20]:
conn.item_exist('123')

data: ["{'embedding': [0.1, 0.2, 0.3], 'document': 'qwerty', 'metadata': {'k1': 'v1'}, 'id': '123'}"]


True

In [19]:
conn.item_exist('123')

True

In [5]:
example = VectorDBInstance(id='1'*32, document='hello world', embedding=np.random.rand(1024))
example1_1 = VectorDBInstance(id='1'*32, document='hello world 4', embedding=np.random.rand(1024))
example2 = VectorDBInstance(id='2'*32, document='hello world 2', embedding=np.random.rand(1024))
example3 = VectorDBInstance(id='3'*32, document='hello world 3', embedding=np.random.rand(1024))

In [6]:
conn.open_connection()
conn.is_open()

True

In [7]:
conn.close_connection()
conn.is_open()

False

In [7]:
conn.create([example, example2, example3])

In [10]:
conn.item_exist(example3.id)

True

In [ ]:
conn.retrieve([example], n_results=2)